In [ ]:
import matplotlib.pyplot as plt 
from misc import *
import tensorflow as tf
from tqdm import tqdm
import os 
from RNN_models import *


%load_ext autoreload
%autoreload 2

In [1]:
from misc import load_data, get_def_path

periods = 5
ppp=1000
defpath = get_def_path()
path = defpath+"{}periods/{}ppp/".format(periods,ppp)


In [2]:
means, covs, signals, coeffs = load_data(path)
tfsignals = tf.convert_to_tensor(signals)[tf.newaxis]
A,dt,C,D = coeffs

FileNotFoundError: [Errno 2] No such file or directory: '/data/uab-giq/scratch/matias/quantera/sanity/trajectories/5periods/1000ppp/0/means.npy'

In [ ]:
times = np.linspace(0,periods*2*np.pi, periods*ppp+1)
total_time = times[-1]
plt.plot(times, means[:,0])

In [ ]:
train_path = path+"/training/train_id_{}/".format(train_id)
os.makedirs(train_path, exist_ok=True)


In [31]:
rmodel = GRNNmodel([C,dt, total_time], cov_in=tf.convert_to_tensor(covs[0].astype(np.float32)))
rmodel.compile(optimizer=tf.keras.optimizers.Adam(lr=0.01))
rmodel.recurrent_layer(tfsignals, initial_state=rmodel.initial_state)
rmodel.trainable_variables[0].assign(tf.convert_to_tensor(A.astype(np.float32)))


history = rmodel.fit(x=tfsignals, y=tfsignals, 
                     epochs = 10**3, callbacks = [tf.keras.callbacks.EarlyStopping(monitor='total_loss', 
                                                                                   min_delta=0, patience=500,
                                                                                   verbose=0,
                                                                                   mode='min')])



histories = rmodel.history.history
keys_histories = list(histories.keys())
for k,v, in histories.items():
    np.save(train_path+"{}".format(k), v, allow_pickle=True)


Epoch 1/1000
1/1 [==============================] - 0s 3ms/step - total_loss: 0.1604 - Coeffs_A: 0.0050 - grads: -0.1252
Epoch 2/1000
1/1 [==============================] - 0s 1ms/step - total_loss: 0.3684 - Coeffs_A: 0.0039 - grads: 0.1719
Epoch 3/1000
1/1 [==============================] - 0s 1ms/step - total_loss: 0.3632 - Coeffs_A: 6.7771e-04 - grads: 0.7894
Epoch 4/1000
1/1 [==============================] - 0s 1ms/step - total_loss: 0.1580 - Coeffs_A: -7.3682e-04 - grads: -0.3193
Epoch 5/1000
1/1 [==============================] - 0s 1ms/step - total_loss: 0.1606 - Coeffs_A: -0.0016 - grads: -0.0911
Epoch 6/1000


KeyboardInterrupt: 